<div style=padding: 30px;
    text-align: center; class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/Images/logocce.png" style="height: 45px;"></a></div>
<div style="float:left;width: 70%;" class='column'><h1 text-align: center>Revisión de contratos con OPENAI</h1></div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div></di

In [1]:
### Librerias de consulta

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import openai
import json
from dotenv import load_dotenv


In [2]:
### Token de OpenAI

OPEN_AI_TOKEN= os.getenv('token_openai')
openai.api_key = OPEN_AI_TOKEN


In [3]:
## Cargamos los datos consultados
Datos_Crudos='../../Datos/Datos Crudos/'
#Consulta = pd.read_parquet(Datos_Crudos+'SECOP_I_II.parquet')
Consulta = pd.read_excel(Datos_Crudos+'Resultado.xlsx')
Consulta.shape

(90693, 28)

In [4]:
Consulta.columns

Index(['Fuente', 'ID Contrato', 'ID Proceso', 'ID entidad Plataforma',
       'Entidad', 'NIT Entidad', 'Orden', 'Modalidad', 'Estado',
       'Descripción proceso', 'Objeto Contrato', 'Tipo de contrato',
       'Fecha de Firma', 'UNSPSC', 'Nombre Proveedor', 'Documento Proveedor',
       'Tipo de documento proveedor', 'Valor del contrato',
       'Departamento Entidad', 'Ciudad Entidad', 'Inicio de contrato',
       'Fin de contrato', 'URL', 'Descripción proceso 2', 'Cluster', 'Tema_x',
       'Sirve_x', 'Tema_y'],
      dtype='object')

In [5]:
import openai
Datos_Procesados='../../Datos/Datos Procesados/'
# Reemplaza 'your_api_key' por tu clave API de OpenAI
openai.api_key = OPEN_AI_TOKEN
for k in range(0,9070):
    print(k)
    ES_EP = []
    Tipo_Contrato = []
    Area=[]
    Sector_Economico=[]
    Proveedor_EP=[]
    Observaciones=[]
    Consulta_1000=Consulta[k*10:(k+1)*10]
    for i in Consulta_1000.index:
        
        Proveedor = Consulta_1000['Nombre Proveedor'][i]
        Entidad = Consulta_1000['Entidad'][i]
        Objeto = Consulta_1000['Descripción proceso'][i]
        Valor = str(Consulta_1000['Valor del contrato'][i])
        # Información inicial proporcionada por el usuario
        info = """
        Se contrató a"""+Proveedor+""" por """+Entidad+""" para """+Objeto+""" por """+Valor+""" pesos colombianos.
        ¿Te parece de Economía Popular?
        Puedes responderme como la fila de una tabla con un encabezado:
        |¿Es Economía Popular?|Tipo de contrato|Área|Sector Económico relacionado|¿El prooveddor es EP?|Observaciones|
        """
        messages = [
                {"role": "system", "content": "Tienes información sobre el contrato."},
                {"role": "user", "content": info},
            ]

            # Realiza la solicitud a la API de OpenAI utilizando el endpoint v1/chat/completions
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=100,  # Ajusta la cantidad de tokens máxima según tus necesidades
            n=1,
            stop=None,
            temperature=0.5,
        )

        answer = response.choices[0]
        answer = answer.to_dict()['message']['content']
        answer=answer.split('|')[1:5]
        try:
            ES_EP.append(answer[0])
        except:
            ES_EP.append('No se')
        
        try:
            Tipo_Contrato.append(answer[1])
        except:
            Tipo_Contrato.append('No se')

        try:
            Area.append(answer[2])
        except:
            Area.append('No se')
            
        try:
            Sector_Economico.append(answer[3])
        except:
            Sector_Economico.append('No se')
        try:
            Proveedor_EP.append(answer[4])
        except:
            Proveedor_EP.append('No se')
        try:
            Observaciones.append(answer[5])
        except:
            Observaciones.append('No se')
    Consulta_1000['Proveedor_EP']=Proveedor_EP
    Consulta_1000['Observaciones']=Observaciones
    Consulta_1000['ES_EP']=ES_EP
    Consulta_1000['Tipo_Contrato']=Tipo_Contrato
    Consulta_1000['Area']=Area
    Consulta_1000['Sector_Economico']=Sector_Economico
    Consulta_1000.to_parquet(Datos_Procesados+'Consulta_'+str(k)+'_10.parquet')
    if k==0:
        Consulta_1000.to_excel(Datos_Procesados+'Consulta_'+str(k)+'_10.xlsx')

0
